## Mengimpor Library

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import wave
import os

## Memuat Audio

In [2]:
file_path = os.path.join(os.getcwd(), 'data', 'ImperialMarch60.wav')

with wave.open(file_path, 'r') as wav_file:
    n_channels = wav_file.getnchannels()
    sampwidth = wav_file.getsampwidth()
    framerate = wav_file.getframerate()
    n_frames = wav_file.getnframes()
    audio_content = wav_file.readframes(n_frames)
    audio_data = np.frombuffer(audio_content, dtype=np.int16)
    
    if n_channels > 1:
        audio_data = audio_data.reshape(-1, n_channels)
        
# trim audio_data from 0 to 30 karena bagian akhir audio memiliki amplitude yang kecil
start_time = 0
end_time = 30

start_sample = int(start_time * framerate)
end_sample = int(end_time * framerate)

audio_data = audio_data[start_sample:end_sample]

## Audio Loudness

Tutorial ini mencakup pengukuran dasar dbFS, RMS, True Peak, dan LUFS untuk analisis audio. Kita akan mengimplementasikan setiap pengukuran langkah demi langkah menggunakan numpy dan pustaka Python dasar.

Selanjutnya, mari kita ubah audio menjadi float32 dan normalisasi:

In [3]:
audio_float = audio_data.astype(np.float32) / np.iinfo(np.int16).max

Penjelasan:
- Kita mengubah data audio menjadi float32 untuk perhitungan yang lebih presisi.
- Kita menormalisasi nilai dengan membagi dengan nilai maksimum yang mungkin untuk integer 16-bit (32767).


### Menghitung dbFS

In [4]:
peak_amplitude = np.max(np.abs(audio_float))
dbfs = 20 * np.log10(peak_amplitude)
print(f"Amplitudo puncak dalam dbFS: {dbfs:.2f}")

Amplitudo puncak dalam dbFS: -3.34


Penjelasan:
- Kita mencari nilai absolut maksimum dalam data audio, yang mewakili amplitudo puncak.
- Kita mengubah ini menjadi desibel menggunakan rumus: 20 * log10(amplitudo).
- Hasilnya adalah amplitudo puncak dalam dbFS.

### Menghitung RMS



In [5]:
squared_audio = np.square(audio_float)
mean_squared = np.mean(squared_audio)
rms = np.sqrt(mean_squared)
rms_db = 20 * np.log10(rms)
print(f"Level RMS: {rms:.4f}")
print(f"Level RMS dalam dB: {rms_db:.2f} dB")

Level RMS: 0.1081
Level RMS dalam dB: -19.32 dB


Penjelasan:
- Kita mengkuadratkan semua nilai dalam data audio.
- Kita menghitung rata-rata dari nilai-nilai yang dikuadratkan ini.
- Kita mengambil akar kuadrat dari rata-rata ini untuk mendapatkan nilai RMS.
- Kita mengubah RMS menjadi desibel menggunakan rumus yang sama seperti sebelumnya.


### Menghitung True Peak

In [6]:
# Upsample dengan faktor 4 menggunakan numpy
audio_upsampled = np.zeros(len(audio_float) * 4)
audio_upsampled[::4] = audio_float
audio_upsampled = np.convolve(audio_upsampled, np.ones(4)/4, mode='same')

true_peak = np.max(np.abs(audio_upsampled))
true_peak_db = 20 * np.log10(true_peak)
print(f"Level True Peak: {true_peak_db:.2f} dB")

Level True Peak: -15.38 dB


Penjelasan:
- Kita membuat array 4 kali ukuran audio asli, menempatkan sampel asli setiap posisi ke-4.
- Kita menggunakan filter rata-rata bergerak sederhana untuk menginterpolasi audio yang di-upsample.
- Kita mencari nilai absolut maksimum dalam audio yang di-upsample ini, yang merupakan True Peak.
- Kita mengubah ini menjadi desibel.

### Menghitung LUFS (disederhanakan)

In [7]:
b = np.array([1.53512485958697, -2.69169618940638, 1.19839281085285])
a = np.array([1.0, -1.69065929318241, 0.73248077421585])
filtered_audio = np.zeros_like(audio_float)
for i in range(len(audio_float)):
    if i < 2:
        filtered_audio[i] = audio_float[i]
    else:
        filtered_audio[i] = (b[0] * audio_float[i] + b[1] * audio_float[i-1] + b[2] * audio_float[i-2]
                             - a[1] * filtered_audio[i-1] - a[2] * filtered_audio[i-2]) / a[0]

# Hitung mean square
ms = np.mean(np.square(filtered_audio))

# LUFS
lufs = -0.691 + 10 * np.log10(ms)
print(f"LUFS: {lufs:.2f}")

LUFS: -18.65


Penjelasan:
- Kita mengimplementasikan filter K-weighting yang disederhanakan menggunakan koefisien yang telah dihitung sebelumnya.
- Kita menerapkan filter ini ke data audio kita menggunakan implementasi dasar dari filter digital.
- Kita menghitung mean square dari audio yang telah difilter.
- Kita menghitung LUFS menggunakan rumus: -0.691 + 10 * log10(mean square).

Perhitungan LUFS yang disederhanakan ini adalah pendekatan dan tidak mencakup semua kompleksitas standar ITU-R BS.1770 resmi, tetapi memberikan gambaran umum tentang tingkat kekerasan suara.


### Ringkasan

In [8]:
print("\nRingkasan pengukuran kekerasan suara:")
print(f"Amplitudo puncak (dbFS): {dbfs:.2f}")
print(f"Level RMS (dB): {rms_db:.2f}")
print(f"Level True Peak (dB): {true_peak_db:.2f}")
print(f"LUFS Terintegrasi: {lufs:.2f}")


Ringkasan pengukuran kekerasan suara:
Amplitudo puncak (dbFS): -3.34
Level RMS (dB): -19.32
Level True Peak (dB): -15.38
LUFS Terintegrasi: -18.65


### Daftar LUFS Standar

- Spotify: -14 LUFS (integrated)
- Apple Music: -16 LUFS (integrated)
- YouTube: -14 LUFS (integrated)
- Tidal: -14 LUFS (integrated)
- Amazon Music: -14 LUFS (integrated)
- Deezer: -15 LUFS (integrated)

---

## Tugas: `ho1`
5. Dengan menggunakan file audio yang anda rekam untuk mengerjakan tugas nomor 4, lakukanlah normalisasi hingga loudness LUFS mencapai -14 LUFS. Berikan penjelasan langkah-langkah yang anda lakukan untuk menyelesaikan tugas ini.

### Ketentuan Penamaan File
```
nim_ho1.ipynb
```

> Saya sudah mendapatkan akses ke server [MOSS Stanford](https://theory.stanford.edu/~aiken/moss/) untuk menggunakan sistem mereka dalam mata kuliah ini untuk mendeteksi code yang "plagiat". Untuk menjamin bahwa code anda tidak terindikasi plagiarisme, silahkan gunakan nama variabel / parameter / dan konfigurasi yang unik dan tidak copy paste. Dan yang terpenting, berusahalah bekerja sendiri. Jika mendapat bantuan baik dari manusia maupun AI / web, segera cantumkan dalam bagian terakhir `ipynb` anda.

### Pengumpulan
- Unggah `.ipynb` ke repositori anda.
- Paste link dari file ke google form pengumpulan tugas
- Export `.ipynb` menjadi `.pdf` dan unggah juga ke google form

Link yang di paste harus mirip dengan format berikut:
```
https://github.com/<username>/<kode_mk>/blob/master/<nim_ho1>.ipynb
```

- *Pastikan repo anda bersifat privat dan anda telah mengundang dosen pengampu ke repositori anda*
- Seluruh tugas untuk ho1 digabung dalam satu file notebook `.ipynb`
- Berikan penjelasan yang sedetail-detailnya untuk setiap bagian eksperimen yang anda lakukan